In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import epanet as epa
import epamodule as em
import seaborn as sns
from cjsbot import CjsBot
from scipy import optimize
from scipy import stats
import time
import os 
import random
from multiprocessing import Process, Lock

In [2]:
t = np.array([0.075, 0.812, 0.317, 0.581, 0.752, 0.994, 0.967, 0.511, 0.851,
              0.925, 0.842, 0.295, 0.633, 0.522, 0.306])
seeds = [661, 308, 769, 343, 491]
posicao_nos = [0.1, 0.3, 0.5, 0.7, 0.9]
q_nos = [10, 20, 30, 40, 50]   
dim=14
seed = seeds[0]
posicao_no = posicao_nos[2]
q_no = q_nos[3]
links = ["../../networks/c-town/nodes", 
        "../../networks/c-town/links", 
        "../../networks/c-town/rede.inp", 
        f"../../networks/c-town/{dim}dim_{posicao_no}_{q_no}.csv"]

diretorio = f'./teste/'
if not os.path.isdir(diretorio):
    os.mkdir(diretorio)
rv = epa.RealValuesNos(links, t[:dim], nos_dim=q_no, posicao=posicao_no)
rv.getRealValue()
rv.close_sim()
net = epa.Rede(links, rv.groups, t[:dim], [0.001,1])

Começando simulação


In [3]:
def selecao(pop, f): 
    random.shuffle(pop)
    pais = []
    i = 0
    while i < len(pop):
        if f(pop[i]) < f(pop[i+1]):
            pais.append(pop[i])
        else:
            pais.append(pop[i+1])
        i += 2
    aux = sorted(pais, key=lambda x: f(x))
    return aux[0], pais

def reproducao(pais, p_mut, p_comb):
    aux = np.copy(pais)
    aux = [list(i) for i in list(aux)]
    filhos = []
    random.shuffle(pais)
    n = 0
    while n <len(pais):
        # acasalamento
        a = pais[n] 
        n+=1
        b = pais[n]
        n+=1
        
        # recombinação
        filho_a = None
        filho_b = None
        # mudar operações 0.8 ~ 1.2
        if np.random.uniform(0,1)<p_comb:
            comb_a = np.random.uniform(0,1, size=(len(a),))
            comb_b = np.random.uniform(0.8, 1.2, size=(len(a),)) - comb_a
            filho_a = a*comb_a + b*comb_b 
            
            comb_a = np.random.uniform(0,1, size=(len(a),))
            comb_b = np.random.uniform(0.8, 1.2, size=(len(a),)) - comb_a
            filho_b = a*comb_a + b*comb_b 
        else:
            filho_a = np.copy(a)
            filho_b = np.copy(b)
            
        # mutação
        if np.random.uniform(0,1) < p_mut:
            filho_a += np.random.normal(0,0.01, size=(len(filho_a),))
        if np.random.uniform(0,1) < p_mut:
            filho_b += np.random.normal(0,0.01, size=(len(filho_a),))
        
        filhos.append(filho_a)
        filhos.append(filho_b)
    return aux+filhos
        

def ga(objetivo, population, gens, p_mut = 0.05, p_comb=0.8, best_score=1e-10):
    population = list(population)
    for i in range(gens):
        x_bsf, pais = selecao(population, objetivo)
        population = reproducao(pais, p_mut, p_comb)
        score = objetivo(x_bsf)
        print(i, score)
        if score < best_score:
            break
    return x_bsf

In [4]:
pop = np.random.random((40,dim))*1+0.001
pop = [list(i) for i in list(pop)]
x = ga(net.objetivo, pop, 1000, p_mut=0.05) # p_mut até 0.08

0 63.60483991991984
1 63.60483991991984
2 63.60483991991984
3 63.60483991991984
4 33.463309652388475
5 33.463309652388475
6 31.953553012124797
7 27.07401464804472
8 22.2329018693026
9 15.249304367259366
10 13.70410649810585
11 13.70410649810585
12 12.24424507711134
13 12.085974997073212
14 12.085974997073212
15 11.036158017677717
16 9.771446535783074
17 9.771446535783074
18 9.771446535783074
19 9.143784703078358
20 9.143784703078358
21 5.714774352122732
22 5.714774352122732
23 5.714774352122732
24 5.714774352122732
25 5.714774352122732
26 5.714774352122732
27 5.714774352122732
28 5.714774352122732
29 5.714774352122732
30 5.714774352122732
31 5.714774352122732
32 5.714774352122732
33 5.714774352122732
34 5.714774352122732
35 5.128641192577545
36 5.128641192577545
37 5.128641192577545
38 5.128641192577545
39 5.128641192577545
40 5.128641192577545
41 5.128641192577545
42 5.128641192577545
43 5.038525906202265
44 5.038525906202265
45 4.840881993658503
46 4.840881993658503
47 4.840881993658

364 1.6765477594690308
365 1.6765477594690308
366 1.6765477594690308
367 1.6765477594690308
368 1.6765477594690308
369 1.6765477594690308
370 1.6765477594690308
371 1.6765477594690308
372 1.6765477594690308
373 1.6765477594690308
374 1.6765477594690308
375 1.6765477594690308
376 1.6765477594690308
377 1.6765477594690308
378 1.6765477594690308
379 1.6765477594690308
380 1.6765477594690308
381 1.6765477594690308
382 1.6765477594690308
383 1.6765477594690308
384 1.6765477594690308
385 1.6765477594690308
386 1.6765477594690308
387 1.6765477594690308
388 1.6765477594690308
389 1.6765477594690308
390 1.6765477594690308
391 1.6765477594690308
392 1.6765477594690308
393 1.6765477594690308
394 1.6765477594690308
395 1.6765477594690308
396 1.6765477594690308
397 1.6765477594690308
398 1.6765477594690308
399 1.6765477594690308
400 1.6765477594690308
401 1.6765477594690308
402 1.6765477594690308
403 1.6765477594690308
404 1.6765477594690308
405 1.3723544290587688
406 1.3723544290587688
407 1.37235

721 0.7507509876368341
722 0.7507509876368341
723 0.7507509876368341
724 0.7507509876368341
725 0.7507509876368341
726 0.7507509876368341
727 0.7507509876368341
728 0.7507509876368341
729 0.7507509876368341
730 0.7507509876368341
731 0.7507509876368341
732 0.7507509876368341
733 0.7507509876368341
734 0.7507509876368341
735 0.7507509876368341
736 0.7507509876368341
737 0.7507509876368341
738 0.7507509876368341
739 0.7507509876368341
740 0.7507509876368341
741 0.7507509876368341
742 0.7507509876368341
743 0.7507509876368341
744 0.7507509876368341
745 0.7507509876368341
746 0.7507509876368341
747 0.7507509876368341
748 0.7507509876368341
749 0.7507509876368341
750 0.7507509876368341
751 0.7507509876368341
752 0.7507509876368341
753 0.7507509876368341
754 0.7507509876368341
755 0.7507509876368341
756 0.7507509876368341
757 0.7507509876368341
758 0.7507509876368341
759 0.7507509876368341
760 0.7507509876368341
761 0.7507509876368341
762 0.7507509876368341
763 0.7507509876368341
764 0.75075

In [74]:
net.get_dist([0.08932101743265615,
 0.6767513972940328,
 0.3240585003027655,
 0.5931063624301887,
 0.8320989978085773,
 0.8986043081338072,
 0.9914785553649216,
 0.48894738635323054,
 0.8089498214808336,
 0.9184494810007497,
 0.8235782531774689,
 0.26448988835512194,
 0.6042284829895238,
 0.4708853639785103])

0.20449261140182226

In [78]:
net.get_dist([0.11076860793043962,
 0.6343247665702868,
 0.3017119295804605,
 0.5876582777101357,
 0.8178843207513453,
 0.8489546471637922,
 0.9960531530275782,
 0.4753525020509169,
 0.8099765912856473,
 0.9069461551064377,
 0.8092465093036442,
 0.23015704558403896,
 0.7116650560965975,
 0.36883040233487174])

0.31233803086667655